In [ ]:

import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score

np.random.seed(42)  # For reproducibility


In [ ]:

# Generate dummy predictions (this would be your model's predicted prices)
y_pred = np.random.rand(100) * 100  # Predicted prices

# Generate dummy actual prices for backtesting (in reality, this would be your test data)
y_test = np.random.rand(99) * 100  # One less because of np.diff


In [ ]:

def calculate_slope(predictions, window=5):
    if len(predictions) < window:
        raise ValueError("Prediction array is shorter than the window size.")
    
    slopes = []
    for i in range(window, len(predictions)):
        x = np.arange(window)
        y = predictions[i-window:i]
        slope, _ = np.polyfit(x, y, 1)
        slopes.append(slope)
    
    return np.array(slopes)

slopes = calculate_slope(y_pred)


In [ ]:

# Set a higher threshold for taking a position
new_threshold = 0.05
new_positions = np.where(slopes > new_threshold, 1, np.where(slopes < -new_threshold, -1, 0))

# Align positions and returns
returns = np.diff(y_test) / y_test[:-1]  # Calculate daily returns
min_length = min(len(new_positions), len(returns))
aligned_positions = new_positions[:min_length]
aligned_returns = returns[:min_length]
aligned_test_prices = y_test[:min_length + 1]  # Align prices

# Backtest with stop-loss
def backtest_strategy_with_stop_loss(positions, test_prices, stop_loss=0.02):
    returns = np.diff(test_prices) / test_prices[:-1]
    positions = positions[:len(returns)]  # Align positions with returns
    
    strategy_returns = []
    for i, ret in enumerate(returns):
        if positions[i] == 1 and ret < -stop_loss:
            strategy_returns.append(-stop_loss)
        elif positions[i] == -1 and ret > stop_loss:
            strategy_returns.append(stop_loss)
        else:
            strategy_returns.append(positions[i] * ret)
    
    cumulative_returns = np.cumprod(1 + np.array(strategy_returns)) - 1
    
    results = pd.DataFrame({
        'Position': positions,
        'Strategy Returns': strategy_returns,
        'Cumulative Returns': cumulative_returns
    })
    
    return results

new_backtest_results = backtest_strategy_with_stop_loss(aligned_positions, aligned_test_prices)


In [ ]:

def evaluate_performance(results):
    total_return = results['Cumulative Returns'].iloc[-1]
    annualized_return = (1 + total_return) ** (252 / len(results)) - 1
    sharpe_ratio = (results['Strategy Returns'].mean() / results['Strategy Returns'].std()) * np.sqrt(252)
    max_drawdown = (results['Cumulative Returns'].cummax() - results['Cumulative Returns']).max()
    
    metrics = {
        'Total Return': total_return,
        'Annualized Return': annualized_return,
        'Sharpe Ratio': sharpe_ratio,
        'Max Drawdown': max_drawdown
    }
    
    return metrics

new_performance_metrics = evaluate_performance(new_backtest_results)
new_performance_metrics
